In [3]:
import numpy as np
from sympy import *
from math import floor
import time

In [4]:
def spoly(f1,f2,order = 'lex'):
    LM1 = LM(f1,order = order)
    LT1 = LT(f1,order = order)
    LM2 = LM(f2,order = order)
    LT2 = LT(f2,order = order)
    numerator = lcm(LM1,LM2)
    S = numerator/LT1*f1 - numerator/LT2 * f2
    return(expand(S))

def minimize_GBasis(G, order = 'lex'):
    for i,g in enumerate(G):
        G[i] = g/LC(g,order = order)
    n = len(G)
    G2 = []
    for g in G:
        H = [f for f in G]
        H.remove(g)
        H = [LT(f,order = order) for f in H]
        LTg = LT(g,order = order)
        q,r = reduced(LTg, H, order = order)
        if not r == 0:
            G2.append(g)
    return(G2)

def reduce_GBasis(G, order = 'lex'):
    G = minimize_GBasis(G, order = order)
    for i,g in enumerate(G):
        H = [f for f in G if not f == g]
        q,r = reduced(g,H, order = order)
        G[i] = r
    return(G)

def GrobAdd_Buchberger(G,f, order = 'lex'):
    Q = [f]
    while len(Q) > 0:
        g = Q.pop(0)
        for g2 in G: 
            S = spoly(g, g2, order = order)
            q,r = reduced(S,G,order = order)
            if not r == 0: 
                Q.append(r)
        G.append(f)
    G = reduce_GBasis(G,order = order)
    return(G)

In [5]:
#Useful functions for both Hypercubes and general Hamming graphs
def read_sets(filename):
    #Read in the sets of a file
    results = []
    with open(filename,'r') as f:
        for line in f.read().splitlines():
            results.append(line.split(','))
    return(results)

def make_linearEqns(A,z):
    #Takes matrix A and variable list z and converts it into a list of linear equations
    z = Matrix(z)
    A = Matrix(A)
    A = A.rref()[0]
    lin_fcns = A*z
    return(list(lin_fcns))

In [6]:
def OneHot(R,k,a,alphabet = None):
    #Converts list of strings to list of one-hot encodings
    if alphabet == None:
        temp = [str(i) for i in range(a)]
        alphabet = ''.join(temp)
    encodings = []
    for r in R:
        encoding = np.zeros((a,k))
        for i in range(k):
            for j,letter in enumerate(alphabet):
                if r[i] == letter:
                    encoding[j,i] = 1
        encodings.append(encoding)
    return(encodings)

def create_polys_improved(k,a):
    #Setup polynomial system for H_k,a without knowing what R is
    variable_string = 'z1'
    for i in range(1,k*a):
        variable_string = variable_string +',z{} '.format(i+1)
    z = var(variable_string)
    P = []
    f = 0
    for i in range(k):
        Pi = []
        func = 0
        func2 = 0
        for j in range(0,a):
            Pi.append(z[i*a+j]*(z[i*a+j]-1)*(z[i*a+j]+1)) #1st condition zi*(zi-1)*(zi+1)
            func= func + z[i*a+j]**2
            func2 = func2 + z[i*a+j]
            f = f + z[i*a+j]**2 #f so that sum(zi**2) neq 0
        func = (2-func)*(func)
        Pi.append(func)
        Pi.append(func2)
        P.append(Pi)
    
    fs = [f-2*i for i in range(1,k+1)]
    return(P,fs,z)

def preprocess_groebner(P,z,k,a):
    G0 = groebner(P[0],order = 'lex')
    G = [g for g in list(G0)]
    for i in range(1,k):
        replacements = [(z[j],z[a*i+j]) for j in range(a)]
        for g in list(G0):
            G.append(g.subs(replacements))
    return(G)

def make_matrix(R,k,a):
    #Converts list of one-hot encodings to the linear system
    temp = [r.flatten('F') for r in R]
    return(np.array(temp,dtype = int))

def check_resolving_improved(R,k,a,alphabet = None):
    (P,fs,z) = create_polys_improved(k,a)
    OH_encodedR = OneHot(R,k,a)
    A = make_matrix(OH_encodedR,k,a)
    lin_fcns = make_linearEqns(A,z)
    pre_G = preprocess_groebner(P,z,k,a)
    G = pre_G
    for f in lin_fcns:
        G = GrobAdd_Buchberger(G,f,order = 'lex')
    for fi in fs:
        Gi = groebner(list(G)+[fi],order = 'lex')
        if not (list(Gi) == [1]):
            return False
    return True

In [8]:
sets = read_sets('Test_Sets/H_3_5.txt')

In [15]:
P,fs,z = create_polys_improved(5,3)
all_P = []
for p in P:
    for f in p:
        all_P.append(f)

In [26]:
R = sets[0]
OH_R = OneHot(R,5,3)
A = make_matrix(OH_R,5,3)
lin_fcns = make_linearEqns(A,z)
print(lin_fcns[0])

z1 + z12 - z13 + 2*z15 + z6 + z8


In [22]:
G = preprocess_groebner(P,z,5,3)
for g in G:
    print(g)

z1 + z2 + z3
z2**3 - z2
z2**2*z3 + z2*z3**2
z3**3 - z3
z4 + z5 + z6
z5**3 - z5
z5**2*z6 + z5*z6**2
z6**3 - z6
z7 + z8 + z9
z8**3 - z8
z8**2*z9 + z8*z9**2
z9**3 - z9
z10 + z11 + z12
z11**3 - z11
z11**2*z12 + z11*z12**2
z12**3 - z12
z13 + z14 + z15
z14**3 - z14
z14**2*z15 + z14*z15**2
z15**3 - z15


In [23]:
G = groebner(all_P,order = 'lex')
for g in list(G):
    print(g)

z1 + z2 + z3
z2**3 - z2
z2**2*z3 + z2*z3**2
z3**3 - z3
z4 + z5 + z6
z5**3 - z5
z5**2*z6 + z5*z6**2
z6**3 - z6
z7 + z8 + z9
z8**3 - z8
z8**2*z9 + z8*z9**2
z9**3 - z9
z10 + z11 + z12
z11**3 - z11
z11**2*z12 + z11*z12**2
z12**3 - z12
z13 + z14 + z15
z14**3 - z14
z14**2*z15 + z14*z15**2
z15**3 - z15


In [25]:
G0 = groebner(list(G)+[lin_fcns[0]],order = 'lex')
for g in list(G0):
    print(g)

z1 + z12 + z14 + 3*z15 + z6 + z8
-z12 - z14 - 3*z15 + z2 + z3 - z6 - z8
z3**3 - z3
2*z12**2*z14*z3*z5 + 6*z12**2*z15*z3*z5 - z12**2*z3**2*z5 - z12**2*z3*z5**2 + 2*z12**2*z3*z5*z8 + 2*z12*z14**2*z3*z5 + 12*z12*z14*z15*z3*z5 - 2*z12*z14*z3**2*z5 - 2*z12*z14*z3*z5**2 + 4*z12*z14*z3*z5*z8 + 18*z12*z15**2*z3*z5 - 6*z12*z15*z3**2*z5 - 6*z12*z15*z3*z5**2 + 12*z12*z15*z3*z5*z8 + z12*z3**2*z5**2 - 2*z12*z3**2*z5*z8 - 2*z12*z3*z5**2*z8 + 2*z12*z3*z5*z8**2 + z12*z3*z5 - z14**2*z3**2*z5 - z14**2*z3*z5**2 + 2*z14**2*z3*z5*z8 + 12*z14*z15**2*z3*z5 - 6*z14*z15*z3**2*z5 - 6*z14*z15*z3*z5**2 + 12*z14*z15*z3*z5*z8 + z14*z3**2*z5**2 - 2*z14*z3**2*z5*z8 - 2*z14*z3*z5**2*z8 + 2*z14*z3*z5*z8**2 + z14*z3*z5 - 9*z15**2*z3**2*z5 - 9*z15**2*z3*z5**2 + 18*z15**2*z3*z5*z8 + 3*z15*z3**2*z5**2 - 6*z15*z3**2*z5*z8 - 6*z15*z3*z5**2*z8 + 6*z15*z3*z5*z8**2 + 19*z15*z3*z5 + z3**2*z5**2*z8 - z3**2*z5*z8**2 - z3*z5**2*z8**2 + z3*z5*z8
-z11**2*z14**2*z3**2*z5**2*z9 - z11**2*z14**2*z3**2*z5*z9**2 - z11**2*z14**2*z3*z5**2*z9

In [27]:
G0 = GrobAdd_Buchberger(list(G),lin_fcns[0])
for g in G0:
    print(g)

KeyboardInterrupt: 